In [59]:
import docker, os

a = %pwd
wd = (a.rsplit('/',2))[0]

GENE = "ORF7a"
BLAST_Results_ncbi=os.path.join("homologue", GENE, "MN908947.3_ORF7a.fasta")
BLAST_Results_uniprot=os.path.join("homologue", GENE, "B20210130A94466D2655679D1FD8953E075198DA8030B5FR.fasta")
REF=os.path.join("reference", GENE, "P0DTC7.fasta")
D_MEME='quay.io/biocontainers/meme:5.3.0--py37pl526h072abfd_0'

WORK_HOST_DIRECTORY=wd
WORK_CONT_DIRECTORY='/opt/project'

In [60]:
client = docker.from_env()
bwa_docker_cont = client.containers.run(
    image=D_MEME,
    volumes={WORK_HOST_DIRECTORY: {'bind': WORK_CONT_DIRECTORY,'mode': 'rw'}},
    working_dir= WORK_CONT_DIRECTORY+'/data/motif/',
    entrypoint="bash",
    command='{0}/scripts_bash/meme.sh {1} {2} {3} {4}'.format(WORK_CONT_DIRECTORY, BLAST_Results_ncbi, BLAST_Results_uniprot, REF, GENE))

In [61]:
from Bio import motifs

with open(os.path.join(wd, "data/motif", GENE, "meme/meme.xml")) as file:
    meme_record = motifs.parse(file, "MEME")

In [62]:
from scripts_python.vcf_gff import gff2rec

motif_in_ref = gff2rec(os.path.join(wd,"data/motif", GENE, "fimo/fimo.gff"))
for seq in motif_in_ref:
    for feature in seq.features:
        print(feature)

type: sequence_motif
location: [14:64](+)
id: CELYHYQECVRGTTVLLKEPCPSGTYEGNSPFHPLADNKFALTCTSTHFA-MEME-1-1-sp|P0DTC7|NS7A_SARS2
qualifiers:
    Key: Alias, Value: ['MEME-1']
    Key: ID, Value: ['CELYHYQECVRGTTVLLKEPCPSGTYEGNSPFHPLADNKFALTCTSTHFA-MEME-1-1-sp|P0DTC7|NS7A_SARS2']
    Key: Name, Value: ['CELYHYQECVRGTTVLLKEPCPSGTYEGNSPFHPLADNKFALTCTSTHFA_sp|P0DTC7|NS7A_SARS2+']
    Key: pvalue, Value: ['5.87e-61']
    Key: qvalue, Value: ['4.23e-59']
    Key: score, Value: ['602']
    Key: sequence, Value: ['CELYHYQECVRGTTVLLKEPCSSGTYEGNSPFHPLADNKFALTCFSTQFA']
    Key: source, Value: ['fimo']

type: sequence_motif
location: [64:93](+)
id: FACADGTRHTYQLRARSVSPKLFIRQEEV-MEME-2-1-sp|P0DTC7|NS7A_SARS2
qualifiers:
    Key: Alias, Value: ['MEME-2']
    Key: ID, Value: ['FACADGTRHTYQLRARSVSPKLFIRQEEV-MEME-2-1-sp|P0DTC7|NS7A_SARS2']
    Key: Name, Value: ['FACADGTRHTYQLRARSVSPKLFIRQEEV_sp|P0DTC7|NS7A_SARS2+']
    Key: pvalue, Value: ['2.18e-34']
    Key: qvalue, Value: ['2.03e-32']
    Key: score,

In [63]:
from Bio import motifs

with open(os.path.join(wd, "data/motif", GENE, "mast/mast.xml")) as f:
    mast_records = motifs.parse(f, "MAST")

for motif in mast_records:
    print(motif.num_occurrences)

0
0
0
0
0
